### 네이버 지도 크롤링
- selenimum을 활용한 크롤링 : 홍대맛집 정보 수집
- iframe
- ajax


In [251]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# 요소에서 텍스트나 속성을 제어하는 것은 힘들다. -> BeautifulSoup 활용
from bs4 import BeautifulSoup as bs
import time


# 0. setting
driver = webdriver.Chrome()
url = "https://map.naver.com/"
driver.get(url)  # 요청이 완료되면 driver 안에 html 문서가 포함되어 있다.(driver==브라우저)
inputSearch = driver.find_element(By.CLASS_NAME, "input_search")  # .붙이면 X
inputSearch.send_keys("홍대맛집")
inputSearch.send_keys(Keys.ENTER)


### iframe
- iframe은 html 문서 안에서 다른 html을 불러오는 또 다른 창이다.
- 보통 같은 서버 내의 다른 문서를 불러오는데 사용한다.
- 한 브라우저에 두 개의 창이 있는 형상
- 네이버 맵의 메인 html은 지도이고 검색하는 창이 iframe으로 되어 있다.
- selenium은 iframe으로 불러온 html을 로딩할 수 있다.

In [252]:
# 1. iframe이 로딩되지 않았는데 찾고 바꾸려고하면 오류 발생
# 네이버 맵은 검색하지 않으면 iframe이 존재하지 않는다

# 1)  원시적인 방법 : 로딩되는 시간을 어림잡아서 계산(오류를 발생할 가능성이 존재, 연산시간이 늘어남)
# import time
# time.sleep(5)


# 2) # 10이내에 로딩이 완료되면 다음 코드 실행 -> iframe이 로딩될때는 완전한 방법이 될 수 없다
driver.implicitly_wait(10)  


# 3) 특정요소가 로딩될 때까지 기다린다
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC

wait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, "searchIframe")))  # searchIframe이 로딩 될 때까지 기다린다.(인자를 튜플로 받음)



[<selenium.webdriver.remote.webelement.WebElement (session="7c0ec78e0de3cdbc7ecfa258f6e4e4b9", element="f.4956FD484946C1C9ACD2A6903DCEFC7F.d.C16AA13B1CE0ED6EE6461788CD9D7769.e.157")>]

In [253]:
# 2. 브라우저 내의 iframe을 id or name 속성으로 찾는다.
searchIframe = driver.find_element(By.ID, "searchIframe")
driver.switch_to.frame(searchIframe)  # 기준으로 삼고 있던 창을 바꾸겠다.

soup = bs(driver.page_source, "html.parser") # 브라우저가 출력하고 있는 html 소스


In [254]:
# <a href="#" target="_self" role="button" class="eUTV2" aria-disabled="true"></a> aria-disabled가 true가 될 때까지 반복문을 돌린다

# 3. 음식점 정보 추출
shopNodes = []  # 음식점 노드리스트

# 1) 첫페이지
shopNodes += soup.select(".UEzoS.rTjJo")
nextPageBtn = driver.find_element(By.CSS_SELECTOR, "div.zRM9F>a:last-child")
# aria-disabled 속성 값이 false면 next 버튼 활성화 / true 비활성화
# **html 내부의 속성 값은 무조건 문자열

# 2) 나머지 페이지
page = 1
while nextPageBtn.get_attribute("aria-disabled") == "false":
    nextPageBtn.click()  # 요소를 마우스로 누르는 것

    # driver(브라우저)에 다음 페이지 정보가 들어온다.
    # 페이징을 동적으로 하고 있다.
    time.sleep(3)  # ajax 페이징 정보(html)를 불러와서 div에 로딩중(iframe과 유사)
    shopNodes += soup.select(".UEzoS.rTjJo")

    page += 1
    print(f'{page}페이지 입니다.')


# 3) 데이터 추출
titles = []
points = []
states = []
categories = []
for shop in shopNodes:
        # title
        titles.append(shop.select_one(".TYaxT").text)

        # point
        point_tmp = shop.select_one(".orXYY")
        point = point_tmp.text.replace("별점", "") if point_tmp != None else "평점 없음"
        points.append(point)

        # category
        categories.append(shop.select_one(".KCMnt").text)
        
        # state
        states.append(shop.select_one(".MVx6e > .MqNOY").text)



2페이지 입니다.
3페이지 입니다.
4페이지 입니다.
5페이지 입니다.


In [255]:
# 4. dataframe으로 표현
import pandas

data = {"상호명": titles, "별점": points, "카테고리": categories, "상태": states}
result = pandas.DataFrame(data)
result

,상호명,별점,카테고리,상태
0,솥돈 홍대점,평점 없음,돼지고기구이,영업 중
1,디어리스트 연남,평점 없음,양식,영업 중
2,홍대공감,평점 없음,이자카야,영업 중
3,고고갈비 홍대상수점,4.65,"육류,고기요리",영업 중
4,피자네버슬립스 합정상수점,4.57,피자,영업 중
5,스케줄합정,4.39,양식,영업 중
6,홍스쭈꾸미 홍대본점,4.43,주꾸미요리,영업 중
7,카페 공명 연남점,4.5,"카페,디저트",영업 중
8,작당모의,4.39,"카페,디저트",영업 중
9,라 룬 비올렛,4.81,양식,브레이크타임
